# Installation

## Install the testory tool and define an alias for it

In [174]:
!wget -nc https://github.com/TestoryTech/examples/releases/download/0.6.5/testory.sh
!wget -nc https://github.com/TestoryTech/examples/releases/download/0.6.5/TestoryTool-101.uber.jar
!chmod +x testory.sh
%alias testory /content/testory.sh

File ‘testory.sh’ already there; not retrieving.

File ‘TestoryTool-101.uber.jar’ already there; not retrieving.



## Install Selenium and run a standalone server

In [175]:
!wget -nc https://github.com/SeleniumHQ/selenium/releases/download/selenium-4.1.0/selenium-server-4.1.2.jar
!apt-get update
!apt-get install chromium-chromedriver

In [176]:
%%script bash --bg
java -jar /content/selenium-server-4.1.2.jar standalone --port 4444 --override-max-sessions true --max-sessions 40  --log /content/selenium.log

# Test model

In [177]:
# A local Magento sever. 
# Useful when connecting to a colab runtime that runs the magento store.
#%env URL http://localhost 

# A public Magebto server.
# Note that the server reboots at round hours and is not available for few minutes at these times.
%env URL = https://magento2-demo.magebit.com/

env: URL=https://magento2-demo.magebit.com/


In [178]:
%%writefile tests.story.js 
//%%js
story('My first end-to-end test', function () {
    with (new SeleniumSession().start(getenv("URL"))) {
        login({username: 'roni_cost@example.com', password: 'roni_cost3@example.com'});

        addToCart({category: 'Women', subCategory: 'Tops', subSubCategory: 'Jackets', product: 'Stellar Solar Jacket', options: ['S', 'Red'], quantity: 3});

        addToCart({category: 'Men', subCategory: 'Tops', subSubCategory: 'Jackets', product: 'Kenobi Trail Jacket', options: ['M', 'Black'], quantity: 3});

        removeFromCart({product: 'Kenobi Trail Jacket'});

        checkExistenceOfProductInCart({product: 'Stellar Solar Jacket' });

        checkOut({shippingMethod: "Fixed",
                  verifyItems: ['Stellar Solar Jacket'], 
                  verifyNonexistenceOfItems: ['Kenobi Trail Jacket']});
    }
});

Overwriting tests.story.js


### Event Definition

The story we entered in the cell above use events such as `AddToCart` and `Login` that are specific to testing the Magento store. This is an example of how testory allows for separation of concerns. The business logic is defined above while the delails of the events are encaspulated in an event definition file.

Expand this section to see the event definition file. Each event is defined using the `define_event` function whose first parameter is the name of the event and second parameter is a callback function that translates the event to Selenium commands. The callback function takes two parameters: a `session` in which the event is invoked and the `event` itself (the `data` field of the BEvent object).


In [179]:
%%writefile events.js 
//%%js
/* @Testory summon selenium */

/***********************************************************************************
 * Login to the store as a regular user.
 *
 * Parameters:
 *   username: string - The user that logs in
 *   password: string - The password of that user
 ************************************************************************************/
define_event("Login", function (session, event) {
    with (session) {
        click("//a[contains(text(),'Sign In')]");
        writeText('//input[@id="email"]', event.username);
        writeText('//input[@id="pass"]', event.password);
        click('//button[@id="send2"]');

        if (event.expectedWelcome)
            waitForVisibility("//span[text()='" + event.expectedWelcome + "']", 10)
    }
})


/***********************************************************************************
 * Login to the store as an admin user.
 *
 * Parameters:
 *   username: string - The user that logs in
 *   password: string - The password of that user
 ************************************************************************************/
define_event("AdminLogin", function (session, event) {
    with (session) {
        writeText('//input[@id="username"]', event.username);
        writeText('//input[@id="login"]', event.password);
        click("//span[text()='Sign in']");
    }
});

/***********************************************************************************
 * Logout a regular user.
 *
 ************************************************************************************/
define_event("Logout", function (session, event) {
    with (session) {
        click("//span[@class='customer-name']//button");
        click("//a[normalize-space()='Sign Out']");
    }
});

/***********************************************************************************
 * Register a  user.
 *
 * Parameters:
 *   s: string              - The name of the session in which we want this event to take place
 *   firsntame : string     - The name of the new user
 *   lastname : string      - The surname of the new user
 *   email_address : string - An email address for the user. Must be unique.
 *   password : string      - Password for the new user.
 ************************************************************************************/
define_event("Register", function (session, event) {
    with (session) {
        click("//a[@href='http://localhost/customer/account/create/']");
        writeText('//input[@id="firstname"]', event.firstname);
        writeText('//input[@id="lastname"]', event.lastname);
        writeText('//input[@id="email_address"]', event.email_address);
        writeText('//input[@id="password"]', event.password);
        writeText('//input[@id="password-confirmation"]', event.password);
        click('//button[@type="submit" and contains(concat(" ",normalize-space(@class)," ")," action ") and contains(concat(" ",normalize-space(@class)," ")," submit ")]');
        assertText("//div[@data-ui-id='message-success']//div[1]", "Thank you for registering with Main Website Store.")
    }
});


/***********************************************************************************
 * Add an item to the cart of the currently logged-in user.
 *
 * Parameters:
 *   s: string                  - The name of the session in which we want this event to take place.
 *   category : string          - The category of the product that we want to add.
 *   subCategory : string       - The sub-category of the product that we want to add.
 *   product : string           - The  product that we want to add.
 *   options : array of strings - A list of options for the product.
 *   quantity: number, optional - The number of items to add.
 ************************************************************************************/
define_event("AddToCart", function (session, event) {
    with (session) {

        click("//span[text()='" + event.category + "']");
        click("(//span[text()='" + event.category + "'])/following::span[text()='" + event.subCategory + "']/following::a[text()[normalize-space()='" + event.subSubCategory + "']]");

        selectByValue("(//span[text()='Show'])[2]/following::select[@class='limiter-options']", '36')

        click("(//img[@alt='" + event.product + "'])[last()]");

        for (let opt of event.options) {
            // Click the options
            click("//div[@data-option-label='" + opt + "']");

            // Verify that it was selected
            waitForVisibility("//div[@data-option-label='" + opt + "' and contains(@class,'selected')]");
        }
        if (event.quantity) {
            writeText("//input[@title='Qty']", event.quantity, true);
        }
        click("//button[@id='product-addtocart-button']/span");
        waitForVisibility("//div[@data-ui-id='message-success']//div[1]");
        assertText("//div[@data-ui-id='message-success']//div[1]", "You added " + event.product + " to your shopping cart.");
    }
})


/***********************************************************************************
 * Remove an item from the cart of the currently logged-in user.
 *
 * Parameters:
 *   s: string        - The name of the session in which we want this event to take place.
 *   product : string - The  product that we want to remove.
 ************************************************************************************/
define_event("RemoveFromCart", function (session, event) {
    with (session) {
        click("//a[@class='action showcart']");
        click("//a[text()[normalize-space()='" + event.product + "']]/following::a[@class='action delete']");
        click("//div[text()='Are you sure you would like to remove this item from the shopping cart?']/following::span[text()='OK']");
        waitForInvisibility("//div[contains(@class,'block block-minicart')]//img[@alt='" + event.product + "']",10);
        click("//button[@id='btn-minicart-close']");
    }
});

/***********************************************************************************
 * Check that a product exists in the cart of the currently logged-in user.
 *
 * Parameters:
 *   s: string -      - The name of the session in which we want this event to take place.
 *   product : string - The  product that we want to remove.
 ************************************************************************************/
define_event("CheckExistenceOfProductInCart", function (session, event) {
    with (session) {
        click("//a[@class='action showcart']");
        waitForVisibility("//div[contains(@class,'block block-minicart')]//img[@alt='" + event.product + "']");
        click("//button[@id='btn-minicart-close']");
    }
});

/***********************************************************************************
 * Check-out the items in the cart of the currently logged-in user.
 *
 * Parameters:
 *   s: string                                              - The name of the session in which we want this event to take place.
 *   verifyItems : array of strings, optional               - A list of items that we expect to see in the cart.
 *   verifyNonexistenceOfItems : array of strings, optional - A list of items that we expect not to see in the cart.
 *   shippingMethod : string, optional                      - The shopping method that we want to use for this order.
 ************************************************************************************/
define_event("CheckOut", function (session, event) {
    with (session) {
        click("//a[@class='action showcart']");
        click("//button[@title='Proceed to Checkout']");

        if (event.verifyItems || event.verifyNonexistenceOfItems) {
            waitForClickability("//div[contains(@class,'items-in-cart')]//div", 20);
            click("//div[contains(@class,'items-in-cart')]//div");
        }

        if (event.verifyItems) {
            for (item of event.verifyItems) {
                waitForVisibility("//img[@alt='" + item + "']");
            }
        }

        if (event.verifyNonexistenceOfItems) {
            for (item of event.verifyNonexistenceOfItems) {
                waitForInvisibility("//img[@alt='" + item + "']", 5);
            }
        }

        if (event.shippingMethod) {
            waitForClickability("//td[text()='" + event.shippingMethod + "']", 5);
            click("//td[text()='" + event.shippingMethod + "']");
        }

        click("//span[text()='Next']");

        if (event.verifyItems) {
            for (item of event.verifyItems) {
                waitForVisibility("//img[@alt='" + item + "']");
            }
        }
        if (event.verifyNonexistenceOfItems) {
            for (item of event.verifyNonexistenceOfItems) {
                waitForInvisibility("//img[@alt='" + item + "']", 5);
            }
        }


        waitForClickability("//button[contains(@class,'action primary')]", 20);
        runCode("jQuery(document.querySelectorAll('button[class*=\"action primary\"]')).click()");
        // click("//button[contains(@class,'action primary')]");
        waitForVisibility("//p[text()='Your order number is: ']", 5);
        click("//span[text()='Continue Shopping']");
    }
});

Overwriting events.js


# Run


In [180]:
testory --verbose run --show --actiondelay 20000 .

  /\
 /XX\                           
(XXXX#####################################
 \XX/  _____  ____  __  _____  ___   ___  _
  \/    | |  | |_  ( (`  | |  / / \ | |_) \ \_/
        |_|  |_|__ _)_)  |_|  \_\_/ |_| \  |_|

[SETUP] FINE Version: 0.6.5-SNAPSHOT
[SETUP] INFO Using tests from path: /content
[SETUP] FINE Run mode: Execute
[EXEC ] INFO Preparing to run
[EXEC>BUILD] FINE Building BProgram Model...
[EXEC>BUILD] FINE Libraries in use:
[EXEC>BUILD] FINE  - Selenium
[EXEC>BUILD] FINE Done
[EXEC ] FINE B-program starting
[EXEC ] FINE Added b-thread: My first end-to-end test
[EXEC ] INFO B-program started
[EXEC ] INFO Selected: [BeginStory-My first end-to-end test {JS_Obj lib:"bp-base"}]
[EXEC ] INFO Selected: [StartSession [id=My first end-to-end test, xpath=https://magento2-demo.magebit.com/]]
Apr 23, 2022 11:59:00 AM org.openqa.selenium.remote.tracing.opentelemetry.OpenTelemetryTracer createTracer
INFO: Using OpenTelemetry for tracing
Apr 23, 2022 11:59:01 AM org.openqa.selenium.r

## Generating and viewing a report

Testory supports various types of reports. In this example, we use the command below to generate a report that details the steps of the tests that we have executed.

In [181]:
testory report TestLog .

  /\
 /XX\                           
(XXXX#####################################
 \XX/  _____  ____  __  _____  ___   ___  _
  \/    | |  | |_  ( (`  | |  / / \ | |_) \ \_/
        |_|  |_|__ _)_)  |_|  \_\_/ |_| \  |_|

[SETUP] INFO Using tests from path: /content
[REPORT] INFO Generating report in /content/./report/extent
[REPORT] INFO Done creating extent report


The generated report can be displayed by clicking on the link below.

In [182]:
testory report Extent .

  /\
 /XX\                           
(XXXX#####################################
 \XX/  _____  ____  __  _____  ___   ___  _
  \/    | |  | |_  ( (`  | |  / / \ | |_) \ \_/
        |_|  |_|__ _)_)  |_|  \_\_/ |_| \  |_|

[SETUP] INFO Using tests from path: /content


In [ ]:
from google.colab import output
get_ipython().system_raw('python3 -m http.server 8888 --directory /content/report&') 
output.serve_kernel_port_as_window(8888, path="extent/html/index.html")